# Materials associated with the paper: 

Cao, W., Zahirovic, S., Flament, N., Williams, S., Golonka, J., and Müller, R. D., 2017: Improving global paleogeography since the late Paleozoic using paleobiology, Biogeosciences, 14, 5425-5439. https://doi.org/10.5194/bg-14-5425-2017.

### This Jupyter notebook is used to (1) convert fossil data into shapefile format, (2) reconstruct the data back in time using plate tectonic models

The code in this notebook is written in Python 2.7. It utilizes standard scientific Python modules and the open source module 'pygplates' which provides a Python API to the open source GPlates software (https://www.gplates.org/). Here, the source code has been modified to Python 3.7 by Behnam Sadeghi.

## Import the Python modules

In [1]:
import pygplates
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Read the input data

In [2]:
# Read the fossil data
pbdb = pd.read_csv('../Data/Fossil/2500Ma-collections.csv',encoding='latin1')
pbdb

,collection_no,collection.authorizer,license,collection.reference_no,latdec,lngdec,period,epoch,stage,ma_max,ma_min,lithology1,lithology2,environment,tectonic_setting,geology_comments
0,184,A. Miller,CC BY,13,79.130000,-76.00000,Cambrian,Early/Lower Cambrian,NaN,530.000,513.0000,"""limestone""",NaN,peritidal,NaN,"""thin bedded, glauconitic, arenaceous lst. gre..."
1,186,J. Sepkoski,CC BY,13,52.833333,-119.25000,Cambrian,NaN,NaN,541.000,485.4000,"""limestone""",NaN,"reef, buildup or bioherm",NaN,"""oolitic, sandly ls, thick bedded, carbonate b..."
2,187,J. Sepkoski,CC BY,13,52.833333,-119.25000,Cambrian,Middle Cambrian,NaN,513.000,505.0000,"""limestone""",NaN,"reef, buildup or bioherm",NaN,"""ls, partially x-bedded; some intraformational..."
3,189,J. Sepkoski,CC BY,13,42.410556,-73.52500,Cambrian,Early/Lower Cambrian,NaN,516.000,513.0000,"""limestone""","""shale""",offshore shelf,NaN,"""sh and ls; ls beds up to 15 cm thick, fine gr..."
4,193,J. Sepkoski,CC BY,13,65.000000,-111.50000,Cambrian,Early/Lower Cambrian,NaN,530.000,513.0000,"""limestone""","""shale""",slope,NaN,"""calcareous sh and silty ls. 'basin facies' -u..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77870,181313,P. Barrett,CC BY,60296,41.700000,-109.00000,Paleogene,Eocene,Ypresian,55.800,50.3000,siltstone,sandstone,channel lag,NaN,NaN
77871,181314,M. Uhen,CC BY,60285,-35.983333,137.05000,Quaternary,Pleistocene,NaN,0.126,0.0117,not reported,NaN,cave,NaN,NaN
77872,181316,N. Ochsenknecht,CC BY,60259,-40.882960,145.14736,Neogene,Miocene,NaN,23.030,15.9700,not reported,NaN,mire/swamp,NaN,NaN
77873,181318,M. Uhen,CC BY,60302,-28.000000,138.00000,NaN,NaN,NaN,28.400,15.9700,claystone,NaN,lacustrine - large,NaN,NaN


## Convert the data into shapefile format

In [3]:
# Extract data set from the database
recon_time = 45
age_max = 49
age_min = 37

#extract subsets
pbdb_subset = pbdb[(pbdb.ma_max<=age_max) & (pbdb.ma_min>=age_min)] 

# Read the tectonic model of Matthews et al (2016)
static_polygons = '../Data/Matthews++_410-0Ma/StaticPolys/PresentDay_StaticPlatePolygons_Matthews++.shp'
# hence, here we can specify a 'fake' rotiation file to avoid reconstructing the fossils
rotation_filename_for_fossils = '../Data/New_Rotation.rot'
rotation_model = pygplates.RotationModel(rotation_filename_for_fossils)

point_features = []

for index,row in pbdb_subset.iterrows():
    point = pygplates.PointOnSphere(float(row.latdec),float(row.lngdec))
    #print (row.LONG)
    point_feature = pygplates.Feature()
    point_feature.set_geometry(point)
    point_feature.set_shapefile_attribute('Environ',str(row.environment))
    point_feature.set_shapefile_attribute('lithology1',str(row.lithology1))
    point_feature.set_shapefile_attribute('lithology2',str(row.lithology2))
    point_feature.set_shapefile_attribute('coll_no',str(row.collection_no))
    point_feature.set_shapefile_attribute('coll_auth',str(row['collection.authorizer']))
    point_feature.set_shapefile_attribute('coll_refno',str(row['collection.reference_no']))
    point_feature.set_shapefile_attribute('ma_max',row.ma_max)
    point_feature.set_shapefile_attribute('ma_min',row.ma_min)
    point_feature.set_shapefile_attribute('period',str(row.period))
    point_feature.set_shapefile_attribute('epoch',str(row.epoch))
    point_feature.set_shapefile_attribute('stage',str(row.stage))
    point_feature.set_valid_time(age_max,age_min)
    point_features.append(point_feature)

# The partition points function can then be used as before
partitioned_point_features = pygplates.partition_into_plates(static_polygons,
                                                       rotation_model,
                                                       point_features) 

# If partitioned feature has plate id = 0, we throw it away 
partitioned_point_features_not_zero = []
for point_feature in partitioned_point_features:
    if point_feature.get_reconstruction_plate_id() != 0:
        partitioned_point_features_not_zero.append(point_feature)   

feature_collection = pygplates.FeatureCollection(partitioned_point_features_not_zero)
feature_collection.write('Output/ProcessingData_1/PresentDay_Paleobiology_'+str(recon_time)+'Ma.shp')


## Reconstruct the data back in time

In [4]:
# read rotation file
input_rotation_filename_forReconstruction = ['../Data/EB_Model_2016/Global_EB_410-fixed_crossovers.rot']
rotation_model_forReconstruction = pygplates.RotationModel(input_rotation_filename_forReconstruction)

# output data at reconstructed coordinates as a shapefile
recon_time = 45
reconstructed_point_features = []
pygplates.reconstruct(feature_collection,
                      rotation_model_forReconstruction,
                      'Output/ProcessingData_1/Reconstructed_Paleobiology_'+str(recon_time)+'Ma.shp',
                      recon_time)